<a href="https://colab.research.google.com/github/pren1/VAST/blob/master/Merge_splitter_prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -rf VAST/
!git clone https://github.com/pren1/VAST.git
!pip3 install soundfile
!pip3 install librosa

Cloning into 'VAST'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 168 (delta 79), reused 70 (delta 26), pack-reused 0
Receiving objects: 100% (168/168), 53.55 MiB | 10.09 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [0]:
!pip3 install tqdm
!pip3 install matplotlib

In [0]:
from VAST.util import *
import pdb
import pprint
import numpy as np
import IPython
import soundfile as sf
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from multiprocessing import Pool, cpu_count

In [0]:
import librosa

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cat /proc/cpuinfo | grep processor | wc -l

4


In [0]:
# debug_name_list = ['宝鐘マリン', '白上フブキ', '天音かなた', '夏色まつり']
debug_name_list = ['白上フブキ', '夏色まつり']
# read_in_path = "gdrive/My Drive/NEW_VERDICT/"

read_in_path = "gdrive/My Drive/3d_verdict/"
# read_in_path = "pren1_voice_splitter_bucket/3d_verdict/"
# read_in_path = f"{local_path}我的云端硬盘/3d_verdict/"

# save_path = "gdrive/My Drive/Free_RAM_VAST/"

# path_1 = save_path + debug_name_list[1] + "/"
# path_2 = save_path + debug_name_list[3] + "/"

In [0]:
def vad_merge(w):
    intervals = librosa.effects.split(w, top_db=20)
    temp = list()
    for s, e in intervals:
        temp.append(w[s:e])
    return np.concatenate(temp, axis=None)

In [0]:
# data_list = []
# label_list = []
data_dict = {}
sample_rate = 8000

generate_test_data = False

for (index, name) in enumerate(debug_name_list):
  data_list = []
  target_path = read_in_path + name + "/"
  first_level_walk = [target_path + f.path.split('/')[-1] for f in os.scandir(target_path) if f.is_dir()]
  # if generate_test_data:
  #   first_level_walk = first_level_walk[:2]
  # else:
  #   first_level_walk = first_level_walk[2:]
  'later, apply the train/test split here.'
  for single_folder_path in tqdm(first_level_walk):
    all_files = files_in_target_folder(single_folder_path, '.npy')
    for single_npy in all_files:
      single_file_path = single_folder_path + "/" + single_npy
      data = load_data_array_from_npy(single_file_path)
      # data, _ = librosa.effects.trim(data, top_db=20)
      # data = vad_merge(data)
      'data length 300s/8000hz'
      for data_clip_index in range(0, len(data), sample_rate * 3):
        'make sure you get the 1 seconds clipped data'
        current_clip = data[data_clip_index:data_clip_index + sample_rate * 3]
        if len(current_clip) == sample_rate * 3:
          data_list.append(current_clip)
          'corrsponding index here review the source label'
          # label_list.append(index)
  data_dict[name] = np.asarray(data_list)
  print(data_dict[name].shape)

100%|██████████| 33/33 [07:08<00:00, 12.98s/it]


(44400, 24000)


100%|██████████| 26/26 [05:09<00:00, 11.89s/it]


(30900, 24000)


In [0]:
# data_dict = load_data_array_from_npy("gdrive/My Drive/3d_verdict/Static_dict.npy").item()

In [0]:
# def get_all_files(save_path):
#   all_files = files_in_target_folder(save_path, '.npy')
#   range_index = np.arange(len(all_files)//2)

#   data_list = []
#   label_list = []
#   for ID in tqdm(range_index):
#     data = load_data_array_from_npy(save_path + f'data_{ID}.npy')
#     label = load_data_array_from_npy(save_path + f'label_{ID}.npy')
#     data_list.append(data)
#     label_list.extend(label)
#   return np.vstack(data_list), label_list  

In [0]:
# data_dict = {}
# for name in debug_name_list:
#   data_dict[name], _ = get_all_files(save_path + name + "/")
#   save_data_array_as_npy(data_dict[name], f"gdrive/My Drive/Free_RAM_VAST/four_people_data_dict_{name}")

In [0]:
# data_dict = {}
# for name in debug_name_list:
#   data_dict[name] = load_data_array_from_npy(f"gdrive/My Drive/Free_RAM_VAST/four_people_data_dict_{name}.npy")

In [0]:
data_dict

{'夏色まつり': array([[-4.38824631e-02, -8.51969719e-02, -5.36992401e-02, ...,
         -2.34751584e-04,  1.09055974e-04, -6.16125355e-04],
        [-7.08632171e-04, -2.10939208e-04,  6.42982253e-04, ...,
          2.83160979e-08,  2.69284879e-08,  2.56153854e-08],
        [ 1.75609202e-07,  3.44422489e-07,  3.59909762e-07, ...,
         -1.90959666e-02, -5.40862344e-02, -2.18683593e-02],
        ...,
        [-2.39831536e-07, -5.56177895e-07,  6.21722620e-08, ...,
         -2.06556066e-07, -1.88264053e-07,  9.94393048e-08],
        [-1.76505424e-08,  5.53303998e-08,  1.55792605e-08, ...,
          5.69141498e-08,  2.58523983e-07,  3.36148460e-08],
        [ 2.39245992e-07,  5.75013360e-07,  3.98323095e-07, ...,
         -1.62030574e-07,  8.91369183e-08, -1.96988097e-07]], dtype=float32),
 '白上フブキ': array([[-6.0869069e-03,  3.6055092e-02,  7.2832294e-02, ...,
          2.7005184e-03,  1.3216899e-03,  1.4454874e-03],
        [ 1.8129856e-03, -6.4096792e-04,  3.7678354e-04, ...,
         -1.4

In [0]:
# data_1, _ = get_all_files(path_1)
# data_2, _ = get_all_files(path_2)

In [0]:
# data_2.shape

In [0]:
# data_1.shape

In [0]:
# save_data_array_as_npy(data_1, "gdrive/My Drive/Free_RAM_VAST/白上フブキ.npy")
# save_data_array_as_npy(data_2, "gdrive/My Drive/Free_RAM_VAST/夏色まつり.npy")

In [0]:
# data_1 = load_data_array_from_npy("gdrive/My Drive/Free_RAM_VAST/白上フブキ.npy")
# data_2 = load_data_array_from_npy("gdrive/My Drive/Free_RAM_VAST/夏色まつり.npy")

In [0]:
# data_1 = data_1[:10000, :]
# data_2 = data_2[:10000, :]

In [0]:
# adapted from Keith Ito's tacotron implementation
# https://github.com/keithito/tacotron/blob/master/util/audio.py

import librosa
import numpy as np


class Audio():
    def __init__(self):
        self.ref_level_db = 20.0
        self.n_fft = 1200
        self.hop_length = 160
        self.win_length = 400
        self.min_level_db = -100.0

    def wav2spec(self, y):
        D = self.stft(y)
        S = self.amp_to_db(np.abs(D)) - self.ref_level_db
        S, D = self.normalize(S), np.angle(D)
        S, D = S.T, D.T # to make [time, freq]
        return S, D

    def spec2wav(self, spectrogram, phase):
        spectrogram, phase = spectrogram.T, phase.T
        # used during inference only
        # spectrogram: enhanced output
        # phase: use noisy input's phase, so no GLA is required
        S = self.db_to_amp(self.denormalize(spectrogram) + self.ref_level_db)
        return self.istft(S, phase)

    def stft(self, y):
        return librosa.stft(y=y, n_fft=self.n_fft,
                            hop_length=self.hop_length,
                            win_length=self.win_length)

    def istft(self, mag, phase):
        stft_matrix = mag * np.exp(1j*phase)
        return librosa.istft(stft_matrix,
                             hop_length=self.hop_length,
                             win_length=self.win_length)

    def amp_to_db(self, x):
        return 20.0 * np.log10(np.maximum(1e-5, x))

    def db_to_amp(self, x):
        return np.power(10.0, x * 0.05)

    def normalize(self, S):
        return np.clip(S / -self.min_level_db, -1.0, 0.0) + 1.0

    def denormalize(self, S):
        return (np.clip(S, 0.0, 1.0) - 1.0) * -self.min_level_db

In [0]:
audio = Audio()

In [0]:
np.random.uniform(low=0.5, high=3.0)

2.5414397155980826

In [0]:
def mix(s1_target, s2):
  srate = 8000
  # w1, _ = librosa.effects.trim(s1_target, top_db=20)
  # w2, _ = librosa.effects.trim(s2, top_db=20)

  # w1_ratio = np.random.uniform(low=0.4, high=3.0)
  # random.choice([1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0])
  # w2_ratio = np.random.uniform(low=0.4, high=3.0)
  # random.choice([1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0])
  'Before everything, we normalize the w1 and w2'
  # s1_target /= np.max(np.abs(s1_target))
  # s2 /= np.max(np.abs(s2))

  # w1_ratio = np.random.uniform(low=0.1, high=1.4)
  # w1_ratio = np.random.uniform(low=0.5, high=1.5)
  # w2_ratio = np.random.uniform(low=0.5, high=1.5)

  w1 = s1_target * 1.0
  w2 = s2 * 1.0
  # plt.plot(s1_target, label="s1_target")
  # plt.plot(s2, label="s2")
  # plt.legend()
  # plt.show()
  # IPython.display.display(IPython.display.Audio(np.asarray(w1).T, rate=8000))
  # IPython.display.display(IPython.display.Audio(np.asarray(w2).T, rate=8000))
  # pdb.set_trace()
  mixed = w1 + w2
  norm = np.max(np.abs(mixed)) * 1.1
  w1, w2, mixed = w1/norm, w2/norm, mixed/norm
  # w1_norm = np.max(np.abs(w1)) * 1.1
  # w2_norm = np.max(np.abs(w2)) * 1.1
  
  # if norm < 1e-5:
  #   print("skipped normalization")
  #   norm = 1

  # if w1_norm < 1e-5:
  #   print(f"Skip w1: {w1_norm}")
  #   w1_norm = 1

  # if w2_norm < 1e-5:
  #   print(f"Skip w2: {w2_norm}")
  #   w2_norm = 1  

  # w1, w2, mixed = w1/w1_norm, w2/w2_norm, mixed/norm
  # print(np.max(np.abs(mixed)))
  'Not that good, but this is a toy'
  # plt.plot(w1, color = 'orange', label='w1')
  # plt.plot(w2, color = 'cyan', label='w2')
  # plt.show()
  # IPython.display.display(IPython.display.Audio(np.asarray(w1).T, rate=8000))
  # IPython.display.display(IPython.display.Audio(np.asarray(w2).T, rate=8000))
  # IPython.display.display(IPython.display.Audio(np.asarray(mixed).T, rate=8000))
  # pdb.set_trace()

  target_mag, _ = audio.wav2spec(w1)
  # print(np.max(w1))
  # resulted_wav = audio.spec2wav(target_mag, phase)
  # plt.plot(resulted_wav, color = 'orange', label='resulted_wav')
  # plt.show()

  # plt.plot(w1, color = 'cyan', label='resulted_wav')
  # plt.show()
  # pdb.set_trace()

  mixed_mag, mixed_phase = audio.wav2spec(mixed)

  # plt.plot(target_mag)
  # plt.title("target_mag")
  # plt.show()

  # plt.plot(mixed_mag)
  # plt.title("mixed_mag")
  # plt.show()

  # pdb.set_trace()
  return target_mag, mixed_mag, mixed_phase

In [0]:
import random

def generate_data_on_the_fly(batch_size, train_dict):
  target_mag_list = []
  mixed_mag_list = []
  mixed_phase_list = []

  speaker_label = []
  for i in tqdm(range(100)):
    'random select two speakers'
    spk1, spk2 = random.sample(['夏色まつり', '白上フブキ'], 2)
    # spk1 = '夏色まつり'
    # spk2 = '白上フブキ'
    spk1_source_data = train_dict[spk1]
    spk2_source_data = train_dict[spk2]
    
    choose_speaker = debug_name_list.index(spk1)

    spk1_index = np.random.choice(range(len(spk1_source_data)), batch_size//100)
    spk1 = spk1_source_data[spk1_index]
    spk2_index = np.random.choice(range(len(spk2_source_data)), batch_size//100)
    spk2 = spk2_source_data[spk2_index]

    for (s1_target, s2) in zip(spk1, spk2):
        target_mag, mixed_mag, mixed_phase = mix(s1_target, s2)
        target_mag_list.append(target_mag)
        mixed_mag_list.append(mixed_mag)
        mixed_phase_list.append(mixed_phase)
        speaker_label.append(choose_speaker)
  return  np.asarray(target_mag_list), np.asarray(mixed_mag_list), np.asarray(mixed_phase_list), np.asarray(speaker_label)

In [0]:
# !rm -rf "Verdict/"

# target_folder = "Verdict/"
# os.mkdir(target_folder)

# os.mkdir(f"{target_folder}/cycle_train_case/")
# os.mkdir(f"{target_folder}/cycle_val_case/")
# os.mkdir(f"{target_folder}/cycle_test_case/")

In [0]:
target_folder = "gdrive/My Drive/New_train/"

In [0]:
# def train_wrapper(i):
#     target_mag_list, mixed_mag_list, mixed_phase_list, speaker_label = generate_data_on_the_fly(10000, data_dict)
#     save_data_array_as_npy(target_mag_list, f"{target_folder}/cycle_train_case/cycle_target_mag_list_{i}")
#     save_data_array_as_npy(mixed_mag_list, f"{target_folder}/cycle_train_case/cycle_mixed_mag_list_{i}")
#     save_data_array_as_npy(mixed_phase_list, f"{target_folder}/cycle_train_case/cycle_mixed_phase_list_{i}")
#     save_data_array_as_npy(speaker_label, f"{target_folder}/cycle_train_case/cycle_speaker_label{i}")

In [0]:
# cpu_num = 4
# if not generate_test_data:
#   arr = list(range(60))
#   with Pool(cpu_num) as p:
#     r = list(tqdm(p.imap(train_wrapper, arr), total=len(arr)))

In [0]:
generate_test_data = False

In [0]:
if not generate_test_data:
  for i in range(60):
    target_mag_list, mixed_mag_list, mixed_phase_list, speaker_label = generate_data_on_the_fly(3000, data_dict)
    save_data_array_as_npy(target_mag_list, f"{target_folder}/scale_train_case/cycle_target_mag_list_{i}")
    save_data_array_as_npy(mixed_mag_list, f"{target_folder}/scale_train_case/cycle_mixed_mag_list_{i}")
    save_data_array_as_npy(mixed_phase_list, f"{target_folder}/scale_train_case/cycle_mixed_phase_list_{i}")
    save_data_array_as_npy(speaker_label, f"{target_folder}/scale_train_case/cycle_speaker_label{i}")

100%|██████████| 100/100 [01:05<00:00,  1.53it/s]


In [0]:
generate_test_data = True

In [0]:
if generate_test_data:
  target_mag_list, mixed_mag_list, mixed_phase_list, speaker_label = generate_data_on_the_fly(1000, data_dict)
  save_data_array_as_npy(target_mag_list, f"{target_folder}/scale_val_case/cycle_target_mag_list")
  save_data_array_as_npy(mixed_mag_list, f"{target_folder}/scale_val_case/cycle_mixed_mag_list")
  save_data_array_as_npy(mixed_phase_list, f"{target_folder}/scale_val_case/cycle_mixed_phase_list")
  save_data_array_as_npy(speaker_label, f"{target_folder}/scale_val_case/cycle_speaker_label")

100%|██████████| 100/100 [00:22<00:00,  4.54it/s]


In [0]:
if generate_test_data:
  target_mag_list, mixed_mag_list, mixed_phase_list, speaker_label = generate_data_on_the_fly(1000, data_dict)
  save_data_array_as_npy(target_mag_list, f"{target_folder}/scale_test_case/cycle_target_mag_list")
  save_data_array_as_npy(mixed_mag_list, f"{target_folder}/scale_test_case/cycle_mixed_mag_list")
  save_data_array_as_npy(mixed_phase_list, f"{target_folder}/scale_test_case/cycle_mixed_phase_list")
  save_data_array_as_npy(speaker_label, f"{target_folder}/scale_test_case/cycle_speaker_label")

100%|██████████| 100/100 [00:25<00:00,  3.94it/s]


In [0]:
# target_mag_list = []
# mixed_mag_list = []
# mixed_phase_list = []

# for (s1_target, s2) in tqdm(zip(data_1, data_2)):
#   target_mag, mixed_mag, mixed_phase = mix(s1_target, s2)
#   target_mag_list.append(target_mag)
#   mixed_mag_list.append(mixed_mag)
#   mixed_phase_list.append(mixed_phase)

In [0]:
# np.asarray(target_mag_list).shape

In [0]:
# save_data_array_as_npy(target_mag_list, "gdrive/My Drive/Free_RAM_VAST/short_target_mag_list")
# save_data_array_as_npy(mixed_mag_list, "gdrive/My Drive/Free_RAM_VAST/short_mixed_mag_list")
# save_data_array_as_npy(mixed_phase_list, "gdrive/My Drive/Free_RAM_VAST/short_mixed_phase_list")

In [0]:
# target_mag = load_data_array_from_npy("gdrive/My Drive/Free_RAM_VAST/target_mag_list.npy")
# mixed_mag = load_data_array_from_npy("gdrive/My Drive/Free_RAM_VAST/mixed_mag_list.npy")

In [0]:
# target_mag.shape

In [0]:
# target_path = "gdrive/My Drive/Spitter_VTB/"

# batch_size = 512
# save_index = 0
# for i in tqdm(range(0, len(target_mag), batch_size)):
#   target_cut = target_mag[i:i+batch_size, :, :]
#   mixed_cut = mixed_mag[i:i+batch_size, :, :]
#   if len(target_cut) == batch_size:
#     save_data_array_as_npy(target_cut, target_path + f"target_mag/{save_index}")
#     save_data_array_as_npy(mixed_cut, target_path + f"mixed_mag/{save_index}")
#     save_index += 1